In [5]:
import pandas as pd
import numpy as np

In [6]:
import psycopg2

# Connect to PostgreSQL
connection = psycopg2.connect(
    host="localhost",
    database="final_db",
    user="group",
    password="project",
    port="5432"
)

# Test the connection
cursor = connection.cursor()
cursor.execute("SELECT version();")
print("Connected to:", cursor.fetchone())

cursor.close()
connection.close()

Connected to: ('PostgreSQL 17.0 (Debian 17.0-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


In [7]:
#census_data = pd.read_csv("demographic_data.csv")
#vaccination_data = pd.read_csv("us_state_vaccinations_data.csv")
#death_data = pd.read_csv("covid19_death_data.csv")

# New file paths
census_data = pd.read_csv(r"C:\Users\zhatz\Documents\GitHub\Data-Science-Masters\Data Engineering Principles and Practice\final_project\data\demographic_data.csv")
vaccination_data = pd.read_csv(r"C:\Users\zhatz\Documents\GitHub\Data-Science-Masters\Data Engineering Principles and Practice\final_project\data\us_state_vaccinations_data.csv")
death_data = pd.read_csv(r"C:\Users\zhatz\Documents\GitHub\Data-Science-Masters\Data Engineering Principles and Practice\final_project\data\covid19_death_data.csv")

### Cleaning the census data

In [8]:
labels_to_keep = [
    "Male",
    "Female",
    "Under 5 years",
    "5 to 9 years",
    "10 to 14 years",
    "15 to 19 years",
    "20 to 24 years",
    "25 to 34 years",
    "35 to 44 years",
    "45 to 54 years",
    "55 to 59 years",
    "60 to 64 years",
    "65 to 74 years",
    "75 to 84 years",
    "85 years and over",
    "White",
    "Black or African American",
    "American Indian and Alaska Native",
    "Asian",
    "Native Hawaiian and Other Pacific Islander",
    "Some other race",
    "White and Black or African American",
    "White and American Indian and Alaska Native",
    "White and Asian",
    "Black or African American and American Indian and Alaska Native",
]

In [9]:
column_selection = [census_data.columns[0]] + list(census_data.filter(like="Estimate").columns)
row_labels = [i.strip() for i in census_data["Label (Grouping)"]]
census_data["Label (Grouping)"] = row_labels
census_data_filtered = census_data.drop_duplicates(subset="Label (Grouping)")
census_data_filtered = census_data_filtered.loc[census_data_filtered.isin(labels_to_keep).iloc[:,0]]
census_data_melted = census_data_filtered.melt(id_vars="Label (Grouping)", var_name="State", value_name="Population")
stripped = [i.replace("!!Estimate", "") for i in census_data_melted["State"]]
census_data_melted["State"] = stripped
census_data_melted = census_data_melted[~census_data_melted["State"].str.contains("Percent")].reset_index(drop=True)
census_data_melted

Label (Grouping)        State  \
0                                                  Male      Alabama   
1                                                Female      Alabama   
2                                         Under 5 years      Alabama   
3                                          5 to 9 years      Alabama   
4                                        10 to 14 years      Alabama   
...                                                 ...          ...   
1295                                    Some other race  Puerto Rico   
1296                White and Black or African American  Puerto Rico   
1297        White and American Indian and Alaska Native  Puerto Rico   
1298                                    White and Asian  Puerto Rico   
1299  Black or African American and American Indian ...  Puerto Rico   

     Population  
0     2,445,896  
1     2,593,981  
2       290,091  
3       298,540  
4       337,730  
...         ...  
1295    968,144  
1296     29,490  
1297        601  
1298        530  
1299        823  

[1300 rows x 3 columns]

In [53]:
census_data_melted.groupby("State")["Population"].agg("sum").reset_index()

State  Population
0                Alabama    14996596
1                 Alaska     2159488
2                Arizona    20601028
3               Arkansas     8878464
4             California   111534664
5               Colorado    16766622
6            Connecticut    10502888
7               Delaware     2952483
8   District of Columbia     1969269
9                Florida    61690529
10               Georgia    31797634
11                Hawaii     4034510
12                 Idaho     5575965
13              Illinois    36928872
14               Indiana    20136537
15                  Iowa     9445475
16                Kansas     8610420
17              Kentucky    13396471
18             Louisiana    13684138
19                 Maine     4072706
20              Maryland    18190375
21         Massachusetts    20332019
22              Michigan    29759051
23             Minnesota    16914231
24           Mississippi     8786262
25              Missouri    18238453
26               Montana     3276543
27              Nebraska     5777514
28                Nevada     9004131
29         New Hampshire     4111793
30            New Jersey    26860325
31            New Mexico     5824291
32              New York    57848065
33        North Carolina    31139339
34          North Dakota     2303373
35                  Ohio    34966560
36              Oklahoma    11724883
37                Oregon    12411815
38          Pennsylvania    38316465
39           Puerto Rico     8655026
40          Rhode Island     3185313
41        South Carolina    15391447
42          South Dakota     2662331
43             Tennessee    20675457
44                 Texas    82207984
45                  Utah     9781952
46               Vermont     1919591
47              Virginia    25428475
48            Washington    22622577
49         West Virginia     5311365
50             Wisconsin    17401210
51               Wyoming     1705384

### Cleaning the vaccination data

In [10]:
vaccination_data["date"] = pd.to_datetime(vaccination_data["date"])
vaccination_data["year"] = [i.year for i in vaccination_data["date"]]
vaccination_data["month"] = [i.month for i in vaccination_data["date"]]
vaccination_data["day"] = [i.day for i in vaccination_data["date"]]
vaccination_data = vaccination_data.loc[vaccination_data["year"]==2021,:].reset_index(drop=True)
vaccination_data

date location  total_vaccinations  total_distributed  \
0     2021-01-12  Alabama             78134.0           377025.0   
1     2021-01-13  Alabama             84040.0           378975.0   
2     2021-01-14  Alabama             92300.0           435350.0   
3     2021-01-15  Alabama            100567.0           444650.0   
4     2021-01-16  Alabama                 NaN                NaN   
...          ...      ...                 ...                ...   
22931 2021-12-27  Wyoming            679449.0           846065.0   
22932 2021-12-28  Wyoming            681822.0           846065.0   
22933 2021-12-29  Wyoming            683069.0           846565.0   
22934 2021-12-30  Wyoming            684672.0           845705.0   
22935 2021-12-31  Wyoming                 NaN                NaN   

       people_vaccinated  people_fully_vaccinated_per_hundred  \
0                70861.0                                 0.15   
1                74792.0                                 0.19   
2                80480.0                                  NaN   
3                86956.0                                 0.28   
4                    NaN                                  NaN   
...                  ...                                  ...   
22931           321856.0                                47.33   
22932           322399.0                                47.41   
22933           322765.0                                47.46   
22934           323245.0                                47.52   
22935                NaN                                  NaN   

       total_vaccinations_per_hundred  people_fully_vaccinated  \
0                                1.59                   7270.0   
1                                1.71                   9245.0   
2                                1.88                      NaN   
3                                2.05                  13488.0   
4                                 NaN                      NaN   
...                               ...                      ...   
22931                          117.40                 273926.0   
22932                          117.81                 274405.0   
22933                          118.02                 274666.0   
22934                          118.30                 275046.0   
22935                             NaN                      NaN   

       people_vaccinated_per_hundred  distributed_per_hundred  \
0                               1.45                     7.69   
1                               1.53                     7.73   
2                               1.64                     8.88   
3                               1.77                     9.07   
4                                NaN                      NaN   
...                              ...                      ...   
22931                          55.61                   146.19   
22932                          55.71                   146.19   
22933                          55.77                   146.27   
22934                          55.85                   146.12   
22935                            NaN                      NaN   

       daily_vaccinations_raw  daily_vaccinations  \
0                         NaN                 NaN   
1                      5906.0              5906.0   
2                      8260.0              7083.0   
3                      8267.0              7478.0   
4                         NaN              7498.0   
...                       ...                 ...   
22931                     NaN              1270.0   
22932                  2373.0              1209.0   
22933                  1247.0              1129.0   
22934                  1603.0              1035.0   
22935                     NaN              1091.0   

       daily_vaccinations_per_million  share_doses_used  total_boosters  \
0                                 NaN             0.207             NaN   
1                              1205.0             0.2

In [11]:
vaccines_totals_columns = [
    "location",
    "total_vaccinations",
    "total_distributed",
    "people_vaccinated",
    "people_fully_vaccinated"
]

In [12]:
vaccinations_total = vaccination_data[vaccines_totals_columns].groupby("location").agg("sum").reset_index()
vaccinations_total

location  total_vaccinations  total_distributed  people_vaccinated  \
0          Alabama        1.058642e+09       1.579341e+09       6.124859e+08   
1           Alaska        2.033552e+08       2.712679e+08       1.087847e+08   
2   American Samoa        1.506269e+07       1.783169e+07       8.385515e+06   
3          Arizona        2.000037e+09       2.476744e+09       1.079612e+09   
4         Arkansas        7.214194e+08       9.738984e+08       4.081857e+08   
..             ...                 ...                ...                ...   
60        Virginia        2.674010e+09       3.137514e+09       1.455369e+09   
61      Washington        2.128565e+09       2.519534e+09       1.151161e+09   
62   West Virginia        3.784129e+08       5.917917e+08       2.118627e+08   
63       Wisconsin        1.735012e+09       1.908157e+09       9.121179e+08   
64         Wyoming        1.318722e+08       1.690251e+08       7.091148e+07   

    people_fully_vaccinated  
0              4.702885e+08  
1              9.045959e+07  
2              6.701509e+06  
3              8.600008e+08  
4              3.121453e+08  
..                      ...  
60             1.194081e+09  
61             9.609692e+08  
62             1.704521e+08  
63             7.855614e+08  
64             5.820194e+07  

[65 rows x 5 columns]

In [13]:
vaccines_daily_columns = [
    "day",
    "month",
    "location",
    "daily_vaccinations",
    "daily_vaccinations_per_million",
    "people_vaccinated",
    "share_doses_used"
]

vaccination_daily = vaccination_data[vaccines_daily_columns]
vaccination_daily

day  month location  daily_vaccinations  \
0       12      1  Alabama                 NaN   
1       13      1  Alabama              5906.0   
2       14      1  Alabama              7083.0   
3       15      1  Alabama              7478.0   
4       16      1  Alabama              7498.0   
...    ...    ...      ...                 ...   
22931   27     12  Wyoming              1270.0   
22932   28     12  Wyoming              1209.0   
22933   29     12  Wyoming              1129.0   
22934   30     12  Wyoming              1035.0   
22935   31     12  Wyoming              1091.0   

       daily_vaccinations_per_million  people_vaccinated  share_doses_used  
0                                 NaN            70861.0             0.207  
1                              1205.0            74792.0             0.222  
2                              1445.0            80480.0             0.212  
3                              1525.0            86956.0             0.226  
4                              1529.0                NaN               NaN  
...                               ...                ...               ...  
22931                          2194.0           321856.0             0.803  
22932                          2089.0           322399.0             0.806  
22933                          1951.0           322765.0             0.807  
22934                          1788.0           323245.0             0.810  
22935                          1885.0                NaN               NaN  

[22936 rows x 7 columns]

### Cleaning covid death data

In [14]:
ages_to_discard = [
    '0-17 years',
    '18-29 years',
    '30-39 years',
    '40-49 years',
    '50-64 years',
]

death_ages_to_combine = [
    ('Under 1 year', '1-4 years', 'Under 5 years'),
]

census_ages_to_combine = [
    ("5 to 9 years", "10 to 14 years", "5 to 14 years"),
    ("15 to 19 years", "20 to 24 years", "15 to 24 years"),
    ("55 to 59 years", "60 to 64 years", "55 to 64 years"),
] # first and second values are combined and third value is the new name

In [15]:
death_columns = [
    "Year",
    "Month",
    "State",
    "Sex",
    "Age Group",
    "COVID-19 Deaths",
]
death_data = death_data[death_columns]
death_data_2021 = death_data[death_data["Year"]==2021.0]
death_data_2021 = death_data_2021.dropna(subset=["Year", "Month"]).drop("Year", axis=1)
death_data_2021 = death_data_2021[~death_data_2021['Age Group'].isin(ages_to_discard)].reset_index(drop=True)
death_data_2021

Month          State        Sex          Age Group  COVID-19 Deaths
0        1.0  United States  All Sexes           All Ages         105565.0
1        1.0  United States  All Sexes       Under 1 year             10.0
2        1.0  United States  All Sexes          1-4 years              5.0
3        1.0  United States  All Sexes         5-14 years             17.0
4        1.0  United States  All Sexes        15-24 years            137.0
...      ...            ...        ...                ...              ...
23323   12.0    Puerto Rico     Female        45-54 years              0.0
23324   12.0    Puerto Rico     Female        55-64 years              NaN
23325   12.0    Puerto Rico     Female        65-74 years              NaN
23326   12.0    Puerto Rico     Female        75-84 years              NaN
23327   12.0    Puerto Rico     Female  85 years and over              NaN

[23328 rows x 5 columns]

Cleaning the death dataset by removing unneeded age groups and combining others

In [16]:
for count, i in enumerate(death_data_2021["Age Group"]):
    for j in death_ages_to_combine:
        if j[0] == i:
            death_data_2021.loc[count, "Age Group"] = j[2]
        if j[1] == i:
            death_data_2021.loc[count, "Age Group"] = j[2]

In [17]:
# Group by the specified columns and sum the 'value' column
group_by_columns_death = [col for col in death_data_2021.columns if col != 'COVID-19 Deaths']
covid_death_monthly = death_data_2021.groupby(group_by_columns_death, as_index=False)['COVID-19 Deaths'].agg("sum")
covid_death_monthly

Month    State        Sex          Age Group  COVID-19 Deaths
0        1.0  Alabama  All Sexes        15-24 years              0.0
1        1.0  Alabama  All Sexes        25-34 years             10.0
2        1.0  Alabama  All Sexes        35-44 years             49.0
3        1.0  Alabama  All Sexes        45-54 years            116.0
4        1.0  Alabama  All Sexes         5-14 years              0.0
...      ...      ...        ...                ...              ...
21379   12.0  Wyoming       Male        65-74 years              0.0
21380   12.0  Wyoming       Male        75-84 years              0.0
21381   12.0  Wyoming       Male  85 years and over              0.0
21382   12.0  Wyoming       Male           All Ages             41.0
21383   12.0  Wyoming       Male      Under 5 years              0.0

[21384 rows x 5 columns]

In [18]:
covid_death_total = covid_death_monthly.groupby(["State", "Sex", "Age Group"], as_index=False)['COVID-19 Deaths'].agg("sum")
covid_death_total

State        Sex          Age Group  COVID-19 Deaths
0     Alabama  All Sexes        15-24 years              0.0
1     Alabama  All Sexes        25-34 years            140.0
2     Alabama  All Sexes        35-44 years            433.0
3     Alabama  All Sexes        45-54 years            966.0
4     Alabama  All Sexes         5-14 years              0.0
...       ...        ...                ...              ...
1777  Wyoming       Male        65-74 years            134.0
1778  Wyoming       Male        75-84 years            108.0
1779  Wyoming       Male  85 years and over             47.0
1780  Wyoming       Male           All Ages            530.0
1781  Wyoming       Male      Under 5 years              0.0

[1782 rows x 4 columns]

### Final Cleaning to match dataset keys

Cleaning the Census dataset age groups to match the death age groups

In [19]:
for count, i in enumerate(census_data_melted["Label (Grouping)"]):
    for j in census_ages_to_combine:
        if j[0] == i:
            census_data_melted.loc[count, "Label (Grouping)"] = j[2]
        if j[1] == i:
            census_data_melted.loc[count, "Label (Grouping)"] = j[2]


In [20]:
# Specify the columns to group by (all columns except 'value')
group_by_columns = [col for col in census_data_melted.columns if col != 'Population']
census_data_melted['Population'] = census_data_melted['Population'].str.replace(',', '').astype(int)

# Group by the specified columns and sum the 'value' column
census_result = census_data_melted.groupby(group_by_columns, as_index=False)['Population'].agg("sum")
census = census_result.rename(columns={"Label (Grouping)": "Grouping"})
census

Grouping          State  Population
0                          15 to 24 years        Alabama      662308
1                          15 to 24 years         Alaska       96118
2                          15 to 24 years        Arizona      971210
3                          15 to 24 years       Arkansas      407443
4                          15 to 24 years     California     5111372
...                                   ...            ...         ...
1139  White and Black or African American       Virginia      134427
1140  White and Black or African American     Washington       91637
1141  White and Black or African American  West Virginia       19963
1142  White and Black or African American      Wisconsin       58974
1143  White and Black or African American        Wyoming        1336

[1144 rows x 3 columns]

## Final Cleaned Datasets

In [21]:
#display(census_result)
#display(cleaned_vaccines_data)
#display(death_data_2021)


display(census)
display(vaccination_daily)
display(vaccinations_total)
display(covid_death_total)
display(covid_death_monthly)

Grouping          State  Population
0                          15 to 24 years        Alabama      662308
1                          15 to 24 years         Alaska       96118
2                          15 to 24 years        Arizona      971210
3                          15 to 24 years       Arkansas      407443
4                          15 to 24 years     California     5111372
...                                   ...            ...         ...
1139  White and Black or African American       Virginia      134427
1140  White and Black or African American     Washington       91637
1141  White and Black or African American  West Virginia       19963
1142  White and Black or African American      Wisconsin       58974
1143  White and Black or African American        Wyoming        1336

[1144 rows x 3 columns]

day  month location  daily_vaccinations  \
0       12      1  Alabama                 NaN   
1       13      1  Alabama              5906.0   
2       14      1  Alabama              7083.0   
3       15      1  Alabama              7478.0   
4       16      1  Alabama              7498.0   
...    ...    ...      ...                 ...   
22931   27     12  Wyoming              1270.0   
22932   28     12  Wyoming              1209.0   
22933   29     12  Wyoming              1129.0   
22934   30     12  Wyoming              1035.0   
22935   31     12  Wyoming              1091.0   

       daily_vaccinations_per_million  people_vaccinated  share_doses_used  
0                                 NaN            70861.0             0.207  
1                              1205.0            74792.0             0.222  
2                              1445.0            80480.0             0.212  
3                              1525.0            86956.0             0.226  
4                              1529.0                NaN               NaN  
...                               ...                ...               ...  
22931                          2194.0           321856.0             0.803  
22932                          2089.0           322399.0             0.806  
22933                          1951.0           322765.0             0.807  
22934                          1788.0           323245.0             0.810  
22935                          1885.0                NaN               NaN  

[22936 rows x 7 columns]

location  total_vaccinations  total_distributed  people_vaccinated  \
0          Alabama        1.058642e+09       1.579341e+09       6.124859e+08   
1           Alaska        2.033552e+08       2.712679e+08       1.087847e+08   
2   American Samoa        1.506269e+07       1.783169e+07       8.385515e+06   
3          Arizona        2.000037e+09       2.476744e+09       1.079612e+09   
4         Arkansas        7.214194e+08       9.738984e+08       4.081857e+08   
..             ...                 ...                ...                ...   
60        Virginia        2.674010e+09       3.137514e+09       1.455369e+09   
61      Washington        2.128565e+09       2.519534e+09       1.151161e+09   
62   West Virginia        3.784129e+08       5.917917e+08       2.118627e+08   
63       Wisconsin        1.735012e+09       1.908157e+09       9.121179e+08   
64         Wyoming        1.318722e+08       1.690251e+08       7.091148e+07   

    people_fully_vaccinated  
0              4.702885e+08  
1              9.045959e+07  
2              6.701509e+06  
3              8.600008e+08  
4              3.121453e+08  
..                      ...  
60             1.194081e+09  
61             9.609692e+08  
62             1.704521e+08  
63             7.855614e+08  
64             5.820194e+07  

[65 rows x 5 columns]

State        Sex          Age Group  COVID-19 Deaths
0     Alabama  All Sexes        15-24 years              0.0
1     Alabama  All Sexes        25-34 years            140.0
2     Alabama  All Sexes        35-44 years            433.0
3     Alabama  All Sexes        45-54 years            966.0
4     Alabama  All Sexes         5-14 years              0.0
...       ...        ...                ...              ...
1777  Wyoming       Male        65-74 years            134.0
1778  Wyoming       Male        75-84 years            108.0
1779  Wyoming       Male  85 years and over             47.0
1780  Wyoming       Male           All Ages            530.0
1781  Wyoming       Male      Under 5 years              0.0

[1782 rows x 4 columns]

Month    State        Sex          Age Group  COVID-19 Deaths
0        1.0  Alabama  All Sexes        15-24 years              0.0
1        1.0  Alabama  All Sexes        25-34 years             10.0
2        1.0  Alabama  All Sexes        35-44 years             49.0
3        1.0  Alabama  All Sexes        45-54 years            116.0
4        1.0  Alabama  All Sexes         5-14 years              0.0
...      ...      ...        ...                ...              ...
21379   12.0  Wyoming       Male        65-74 years              0.0
21380   12.0  Wyoming       Male        75-84 years              0.0
21381   12.0  Wyoming       Male  85 years and over              0.0
21382   12.0  Wyoming       Male           All Ages             41.0
21383   12.0  Wyoming       Male      Under 5 years              0.0

[21384 rows x 5 columns]

## Database Table Creation
#### This snippet demonstrates how tables are created with primary and foreign keys in PostgreSQL.

In [22]:
print(census.columns)
print(vaccination_daily.columns)
print(vaccinations_total.columns)
print(covid_death_total.columns)
print(covid_death_monthly.columns)

Index(['Grouping', 'State', 'Population'], dtype='object')
Index(['day', 'month', 'location', 'daily_vaccinations',
       'daily_vaccinations_per_million', 'people_vaccinated',
       'share_doses_used'],
      dtype='object')
Index(['location', 'total_vaccinations', 'total_distributed',
       'people_vaccinated', 'people_fully_vaccinated'],
      dtype='object')
Index(['State', 'Sex', 'Age Group', 'COVID-19 Deaths'], dtype='object')
Index(['Month', 'State', 'Sex', 'Age Group', 'COVID-19 Deaths'], dtype='object')


In [23]:
import psycopg2
import pandas as pd

# Verify that each DataFrame has been loaded and contains data
print("Census DataFrame:")
display(census)
print("Vaccination Daily DataFrame:")
display(vaccination_daily)
print("Vaccinations Total DataFrame:")
display(vaccinations_total)
print("COVID Death Total DataFrame:")
display(covid_death_total)
print("COVID Death Monthly DataFrame:")
display(covid_death_monthly)

# Re-establish the connection if it's closed
connection = psycopg2.connect(
    host="localhost",
    database="final_db",
    user="group",
    password="project",
    port="5432"
)

# Create a cursor
cursor = connection.cursor()

# Drop existing tables if they already exist
cursor.execute("""

    CREATE SCHEMA IF NOT EXISTS covid19;

    DROP TABLE IF EXISTS covid19.vaccinations_total, covid19.vaccination_daily, covid19.covid_death_monthly, covid19.covid_death_total, covid19.census;

""")
connection.commit()

# Define SQL to create each table with primary keys and foreign keys
create_census_table = """
CREATE TABLE IF NOT EXISTS covid19.census (
    state VARCHAR(100) PRIMARY KEY,
    grouping VARCHAR(100),
    population INTEGER
);
"""

create_vaccinations_total_table = """
CREATE TABLE IF NOT EXISTS covid19.vaccinations_total (
    location VARCHAR(100) PRIMARY KEY,
    total_vaccinations BIGINT,
    total_distributed BIGINT,
    people_vaccinated BIGINT,
    people_fully_vaccinated BIGINT,
    FOREIGN KEY (location) REFERENCES covid19.census(state)
);
"""

create_vaccination_daily_table = """
CREATE TABLE IF NOT EXISTS covid19.vaccination_daily (
    day INTEGER,
    month INTEGER,
    location VARCHAR(100),
    daily_vaccinations NUMERIC,
    daily_vaccinations_per_million NUMERIC,
    people_vaccinated NUMERIC,
    share_doses_used FLOAT,
    PRIMARY KEY (day, month, location),
    FOREIGN KEY (location) REFERENCES covid19.census(state)
);
"""

create_covid_death_monthly_table = """
CREATE TABLE IF NOT EXISTS covid19.covid_death_monthly (
    month INTEGER,
    State VARCHAR(100),
    Sex VARCHAR(10),
    "Age Group" VARCHAR(50),
    "COVID-19 Deaths" INTEGER,
    PRIMARY KEY (month, State, "Age Group"),
    FOREIGN KEY (State) REFERENCES covid19.census(state)
);
"""

create_covid_death_total_table = """
CREATE TABLE IF NOT EXISTS covid19.covid_death_total (
    State VARCHAR(100),
    Sex VARCHAR(10),
    "Age Group" VARCHAR(50),
    "COVID-19 Deaths" INTEGER,
    PRIMARY KEY (State, Sex, "Age Group"),
    FOREIGN KEY (State) REFERENCES covid19.census(state)
);
"""

# Execute each table creation
cursor.execute(create_census_table)
cursor.execute(create_vaccinations_total_table)
cursor.execute(create_vaccination_daily_table)
cursor.execute(create_covid_death_monthly_table)
cursor.execute(create_covid_death_total_table)
connection.commit()

Census DataFrame:


Grouping          State  Population
0                          15 to 24 years        Alabama      662308
1                          15 to 24 years         Alaska       96118
2                          15 to 24 years        Arizona      971210
3                          15 to 24 years       Arkansas      407443
4                          15 to 24 years     California     5111372
...                                   ...            ...         ...
1139  White and Black or African American       Virginia      134427
1140  White and Black or African American     Washington       91637
1141  White and Black or African American  West Virginia       19963
1142  White and Black or African American      Wisconsin       58974
1143  White and Black or African American        Wyoming        1336

[1144 rows x 3 columns]

Vaccination Daily DataFrame:


day  month location  daily_vaccinations  \
0       12      1  Alabama                 NaN   
1       13      1  Alabama              5906.0   
2       14      1  Alabama              7083.0   
3       15      1  Alabama              7478.0   
4       16      1  Alabama              7498.0   
...    ...    ...      ...                 ...   
22931   27     12  Wyoming              1270.0   
22932   28     12  Wyoming              1209.0   
22933   29     12  Wyoming              1129.0   
22934   30     12  Wyoming              1035.0   
22935   31     12  Wyoming              1091.0   

       daily_vaccinations_per_million  people_vaccinated  share_doses_used  
0                                 NaN            70861.0             0.207  
1                              1205.0            74792.0             0.222  
2                              1445.0            80480.0             0.212  
3                              1525.0            86956.0             0.226  
4                              1529.0                NaN               NaN  
...                               ...                ...               ...  
22931                          2194.0           321856.0             0.803  
22932                          2089.0           322399.0             0.806  
22933                          1951.0           322765.0             0.807  
22934                          1788.0           323245.0             0.810  
22935                          1885.0                NaN               NaN  

[22936 rows x 7 columns]

Vaccinations Total DataFrame:


location  total_vaccinations  total_distributed  people_vaccinated  \
0          Alabama        1.058642e+09       1.579341e+09       6.124859e+08   
1           Alaska        2.033552e+08       2.712679e+08       1.087847e+08   
2   American Samoa        1.506269e+07       1.783169e+07       8.385515e+06   
3          Arizona        2.000037e+09       2.476744e+09       1.079612e+09   
4         Arkansas        7.214194e+08       9.738984e+08       4.081857e+08   
..             ...                 ...                ...                ...   
60        Virginia        2.674010e+09       3.137514e+09       1.455369e+09   
61      Washington        2.128565e+09       2.519534e+09       1.151161e+09   
62   West Virginia        3.784129e+08       5.917917e+08       2.118627e+08   
63       Wisconsin        1.735012e+09       1.908157e+09       9.121179e+08   
64         Wyoming        1.318722e+08       1.690251e+08       7.091148e+07   

    people_fully_vaccinated  
0              4.702885e+08  
1              9.045959e+07  
2              6.701509e+06  
3              8.600008e+08  
4              3.121453e+08  
..                      ...  
60             1.194081e+09  
61             9.609692e+08  
62             1.704521e+08  
63             7.855614e+08  
64             5.820194e+07  

[65 rows x 5 columns]

COVID Death Total DataFrame:


State        Sex          Age Group  COVID-19 Deaths
0     Alabama  All Sexes        15-24 years              0.0
1     Alabama  All Sexes        25-34 years            140.0
2     Alabama  All Sexes        35-44 years            433.0
3     Alabama  All Sexes        45-54 years            966.0
4     Alabama  All Sexes         5-14 years              0.0
...       ...        ...                ...              ...
1777  Wyoming       Male        65-74 years            134.0
1778  Wyoming       Male        75-84 years            108.0
1779  Wyoming       Male  85 years and over             47.0
1780  Wyoming       Male           All Ages            530.0
1781  Wyoming       Male      Under 5 years              0.0

[1782 rows x 4 columns]

COVID Death Monthly DataFrame:


Month    State        Sex          Age Group  COVID-19 Deaths
0        1.0  Alabama  All Sexes        15-24 years              0.0
1        1.0  Alabama  All Sexes        25-34 years             10.0
2        1.0  Alabama  All Sexes        35-44 years             49.0
3        1.0  Alabama  All Sexes        45-54 years            116.0
4        1.0  Alabama  All Sexes         5-14 years              0.0
...      ...      ...        ...                ...              ...
21379   12.0  Wyoming       Male        65-74 years              0.0
21380   12.0  Wyoming       Male        75-84 years              0.0
21381   12.0  Wyoming       Male  85 years and over              0.0
21382   12.0  Wyoming       Male           All Ages             41.0
21383   12.0  Wyoming       Male      Under 5 years              0.0

[21384 rows x 5 columns]

## Data Insertion Into Tables

In [24]:
import warnings

# Suppress specific UserWarning from pandas about SQLAlchemy
warnings.filterwarnings("ignore", category=UserWarning, message="pandas only supports SQLAlchemy connectable")


In [46]:
missing_states_monthly

{'New York City', 'United States'}

In [47]:
missing_states_total

{'New York City', 'United States'}

In [50]:
census

Grouping          State  Population
0                          15 to 24 years        Alabama      662308
1                          15 to 24 years         Alaska       96118
2                          15 to 24 years        Arizona      971210
3                          15 to 24 years       Arkansas      407443
4                          15 to 24 years     California     5111372
...                                   ...            ...         ...
1139  White and Black or African American       Virginia      134427
1140  White and Black or African American     Washington       91637
1141  White and Black or African American  West Virginia       19963
1142  White and Black or African American      Wisconsin       58974
1143  White and Black or African American        Wyoming        1336

[1144 rows x 3 columns]

In [54]:
missing_locations

{'American Samoa',
 'Bureau of Prisons',
 'Dept of Defense',
 'Federated States of Micronesia',
 'Guam',
 'Indian Health Svc',
 'Long Term Care',
 'Marshall Islands',
 'New York State',
 'Northern Mariana Islands',
 'Republic of Palau',
 'United States',
 'Veterans Health',
 'Virgin Islands'}

In [ ]:
# Insert data into the census table
for _, row in census.iterrows():
    cursor.execute("""
        INSERT INTO covid19.census (state, grouping, population)
        VALUES (%s, %s, %s)
        ON CONFLICT DO NOTHING;
    """, (row['State'], row['Grouping'], row['Population']))
connection.commit()

# Identify missing locations in `vaccinations_total`
missing_locations = set(vaccinations_total['location']) - set(census['State'])

# Insert missing locations into `census` table with default values
for location in missing_locations:
    #print(f"Inserting missing location '{location}' into census")
    cursor.execute("""
        INSERT INTO covid19.census (state, grouping, population)
        VALUES (%s, %s, %s)
        ON CONFLICT DO NOTHING;
    """, (location, 'Unknown', 0))
connection.commit()

# Identify missing states in `covid_death_monthly` and `covid_death_total`
missing_states_monthly = set(covid_death_monthly['State']) - set(census['State'])
missing_states_total = set(covid_death_total['State']) - set(census['State'])

# Combine all missing states into a single set
missing_states = missing_states_monthly | missing_states_total

# Insert missing states into `census` table with default values
for state in missing_states:
    #print(f"Inserting missing state '{state}' into census")
    cursor.execute("""
        INSERT INTO covid19.census (state, grouping, population)
        VALUES (%s, %s, %s)
        ON CONFLICT DO NOTHING;
    """, (state, 'Unknown', 0))
connection.commit()

# Insert data into the vaccinations_total table
for _, row in vaccinations_total.iterrows():
    cursor.execute("""
        INSERT INTO covid19.vaccinations_total (location, total_vaccinations, total_distributed, people_vaccinated, people_fully_vaccinated)
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT DO NOTHING;
    """, (row['location'], row['total_vaccinations'], row['total_distributed'], row['people_vaccinated'], row['people_fully_vaccinated']))
connection.commit()

# Insert data into the vaccination_daily table
for _, row in vaccination_daily.iterrows():
    cursor.execute("""
        INSERT INTO covid19.vaccination_daily (day, month, location, daily_vaccinations, daily_vaccinations_per_million, people_vaccinated, share_doses_used)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT DO NOTHING;
    """, (row['day'], row['month'], row['location'], row['daily_vaccinations'], row['daily_vaccinations_per_million'], row['people_vaccinated'], row['share_doses_used']))
connection.commit()

# Insert data into the covid_death_monthly table
for _, row in covid_death_monthly.iterrows():
    cursor.execute("""
        INSERT INTO covid19.covid_death_monthly (month, State, Sex, "Age Group", "COVID-19 Deaths")
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT DO NOTHING;
    """, (row['Month'], row['State'], row['Sex'], row['Age Group'], row['COVID-19 Deaths']))
connection.commit()

# Insert data into the covid_death_total table
for _, row in covid_death_total.iterrows():
    cursor.execute("""
        INSERT INTO covid19.covid_death_total (State, Sex, "Age Group", "COVID-19 Deaths")
        VALUES (%s, %s, %s, %s)
        ON CONFLICT DO NOTHING;
    """, (row['State'], row['Sex'], row['Age Group'], row['COVID-19 Deaths']))
connection.commit()

# Verify data insertion by counting rows in each table
print("\nRow counts after data insertion:")
for table in ["covid19.census", "covid19.vaccinations_total", "covid19.vaccination_daily", "covid19.covid_death_monthly", "covid19.covid_death_total"]:
    query = f"SELECT COUNT(*) FROM {table};"
    count = pd.read_sql_query(query, connection)
    print(f"{table} Table Row Count:", count.iloc[0, 0])

# Close the cursor and connection
cursor.close()
connection.close()


Row counts after data insertion:
covid19.census Table Row Count: 67
covid19.vaccinations_total Table Row Count: 65
covid19.vaccination_daily Table Row Count: 22936
covid19.covid_death_monthly Table Row Count: 7128
covid19.covid_death_total Table Row Count: 1782


## 

## Sample SQL Queries to Verify Data

In [26]:
import psycopg2
import pandas as pd

# Re-open the connection if it's closed
try:
    # If the connection is closed, this will raise an error, and we will reconnect
    connection.cursor()
except (Exception, psycopg2.OperationalError):
    connection = psycopg2.connect(
        host="localhost",
        database="final_db",
        user="group",
        password="project",
        port="5432"
    )

# SQL Query Example: Selecting data from census table
query = "SELECT * FROM covid19.census LIMIT 5;"
census_sample = pd.read_sql_query(query, connection)
print("\033[1mCensus Sample Data:\033[0m")  # Bolded title
print(census_sample)
print()  # Adds a blank line after the census output

# SQL Query Example: Selecting data from vaccinations_total table
query = "SELECT * FROM covid19.vaccinations_total LIMIT 5;"
vaccination_sample = pd.read_sql_query(query, connection)
print("\033[1mVaccination Totals Sample Data:\033[0m")  # Bolded title
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', 1000)       # Set a large width for display
print(vaccination_sample)
print()  # Adds a blank line after the vaccination totals output

# SQL Query Example: Selecting data from covid_death_monthly table
query = "SELECT * FROM covid19.covid_death_monthly LIMIT 5;"
death_sample = pd.read_sql_query(query, connection)
print("\033[1mCOVID-19 Deaths Monthly Sample Data:\033[0m")  # Bolded title
print(death_sample)
print()  # Adds a blank line after the COVID-19 deaths output

# SQL Query Example: Selecting data from vaccination_daily table
query = "SELECT * FROM covid19.vaccination_daily LIMIT 5;"
vaccination_daily_sample = pd.read_sql_query(query, connection)
print("\033[1mVaccination Daily Sample Data:\033[0m")  # Bolded title
print(vaccination_daily_sample)
print()  # Adds a blank line after the vaccination daily output

# SQL Query Example: Selecting data from covid_death_total table
query = "SELECT * FROM covid19.covid_death_total LIMIT 5;"
covid_death_total_sample = pd.read_sql_query(query, connection)
print("\033[1mCOVID-19 Deaths Total Sample Data:\033[0m")  # Bolded title
print(covid_death_total_sample)
print()  # Adds a blank line after the COVID-19 deaths total output


Census Sample Data:
        state        grouping  population
0     Alabama  15 to 24 years      662308
1      Alaska  15 to 24 years       96118
2     Arizona  15 to 24 years      971210
3    Arkansas  15 to 24 years      407443
4  California  15 to 24 years     5111372

Vaccination Totals Sample Data:
         location  total_vaccinations  total_distributed  people_vaccinated  people_fully_vaccinated
0         Alabama          1058642058         1579341470          612485871                470288500
1          Alaska           203355220          271267935          108784678                 90459593
2  American Samoa            15062694           17831690            8385515                  6701509
3         Arizona          2000037375         2476743910         1079611508                860000774
4        Arkansas           721419388          973898435          408185667                312145342

COVID-19 Deaths Monthly Sample Data:
   month    state        sex    Age Group  COVID-19

## Data Exploration and Analysis
#### Analyzing the data by running more specific SQL queries
#### Aggregate Queries: Find summaries, such as total counts, averages, or sums

**Total COVID-19 Deaths by State**

This query calculates the total number of COVID-19 deaths for each state. It groups the data by state and sums the deaths across all months and age groups, ordering the results from highest to lowest.

In [27]:
query = """
SELECT 
    State,
    SUM("COVID-19 Deaths") AS total_deaths
FROM 
    covid19.covid_death_monthly
GROUP BY 
    State
ORDER BY 
    total_deaths DESC;
"""
total_deaths_by_state = pd.read_sql_query(query, connection)
print(total_deaths_by_state)

                   state  total_deaths
0          United States        926526
1                  Texas         97496
2             California         96609
3                Florida         77669
4                   Ohio         41008
5           Pennsylvania         40986
6                Georgia         34537
7         North Carolina         30415
8               Michigan         29789
9               New York         29511
10               Arizona         28026
11             Tennessee         27325
12              Illinois         23492
13               Indiana         20023
14               Alabama         19340
15              Virginia         19232
16            New Jersey         19128
17        South Carolina         18904
18         New York City         18041
19              Missouri         17799
20              Kentucky         17333
21              Oklahoma         15608
22             Louisiana         13738
23              Colorado         12319
24            Washington 

**COVID-19 Deaths by Age Group for California**

This query examines COVID-19 deaths by age group for a specific state (in this case, California). It allows us to see which age groups were most affected by COVID-19 in the state.

In [28]:
query = """
SELECT 
    "Age Group",
    SUM("COVID-19 Deaths") AS total_deaths
FROM 
    covid19.covid_death_monthly
WHERE 
    State = 'California'
GROUP BY 
    "Age Group"
ORDER BY 
    total_deaths DESC;
"""
deaths_by_age_group = pd.read_sql_query(query, connection)
print(deaths_by_age_group)

            Age Group  total_deaths
0            All Ages         48330
1         65-74 years         11235
2         75-84 years         10856
3   85 years and over         10042
4         55-64 years          8874
5         45-54 years          4367
6         35-44 years          1942
7         25-34 years           823
8         15-24 years           140
9          5-14 years             0
10      Under 5 years             0


**Total Vaccinations by State**

This query shows the total number of people vaccinated in each state. It sums up the `people_vaccinated` column for each state, which helps to understand the vaccination coverage across states.

In [29]:
query = """
SELECT 
    location AS State,
    SUM(people_vaccinated) AS total_vaccinated
FROM 
    covid19.vaccinations_total
GROUP BY 
    location
ORDER BY 
    total_vaccinated DESC;
"""
total_vaccinated_by_state = pd.read_sql_query(query, connection)
print(total_vaccinated_by_state)

                       state  total_vaccinated
0              United States      5.103508e+10
1                 California      6.433218e+09
2                      Texas      4.081486e+09
3                    Florida      3.463451e+09
4             New York State      3.426877e+09
..                       ...               ...
60  Northern Mariana Islands      8.968232e+06
61            American Samoa      8.385515e+06
62          Marshall Islands      5.655669e+06
63         Republic of Palau      4.228663e+06
64            Long Term Care      3.210878e+06

[65 rows x 2 columns]


**Monthly COVID-19 Deaths Over Time for New York**

This query retrieves monthly COVID-19 deaths over time for a specific state (in this example, New York). It helps visualize the trend of deaths over time in New York, which can be useful for understanding the progression of the pandemic in that state.

In [30]:
query = """
SELECT 
    month,
    SUM("COVID-19 Deaths") AS total_deaths
FROM 
    covid19.covid_death_monthly
WHERE 
    State = 'New York'
GROUP BY 
    month
ORDER BY 
    month;
"""
monthly_deaths_ny = pd.read_sql_query(query, connection)
print(monthly_deaths_ny)

    month  total_deaths
0       1          8690
1       2          4425
2       3          2300
3       4          1783
4       5          1039
5       6           474
6       7           239
7       8           908
8       9          1706
9      10          1987
10     11          2163
11     12          3797


**COVID-19 Deaths by Gender in Texas**

This query provides a breakdown of COVID-19 deaths by gender for Texas. It groups the data by gender to help analyze any disparities in COVID-19 impact between males and females in this state.

In [31]:
query = """
SELECT 
    Sex,
    SUM("COVID-19 Deaths") AS total_deaths
FROM 
    covid19.covid_death_monthly
WHERE 
    State = 'Texas'
GROUP BY 
    Sex
ORDER BY 
    total_deaths DESC;
"""
deaths_by_gender_tx = pd.read_sql_query(query, connection)
print(deaths_by_gender_tx)

         sex  total_deaths
0  All Sexes         97496


**Average Number of Vaccinations Distributed per State**

This query calculates the average number of vaccine doses distributed per state. This can give insights into the allocation and distribution patterns across different regions.

In [32]:
query = """
SELECT 
    location AS state,
    SUM(total_distributed) AS total_distributed
FROM 
    covid19.vaccinations_total
GROUP BY 
    location
ORDER BY 
    total_distributed DESC;
"""
avg_vaccine_distribution = pd.read_sql_query(query, connection)
print(avg_vaccine_distribution)

                             state  total_distributed
0                    United States       1.124559e+11
1                       California       1.330869e+10
2                            Texas       9.713806e+09
3                          Florida       7.903413e+09
4                   New York State       7.320534e+09
..                             ...                ...
60  Federated States of Micronesia       2.320157e+07
61                  American Samoa       1.783169e+07
62                Marshall Islands       1.572329e+07
63                  Long Term Care       1.365735e+07
64               Republic of Palau       8.043360e+06

[65 rows x 2 columns]


In [33]:
###